In [ ]:
# | default_exp agent.planner.executor

In [2]:
# | export
import json
from typing import Optional
from rich.console import Console

from agentic.core.agent import Agent, AgentConfig
from agentic.configs.loader import get_model_config

from agentic.agent.planner.models import ProjectContext, ExecutionStatus
from agentic.agent.planner.breakdown import ProjectBreakdownGenerator
from agentic.agent.planner.task_generator import TaskGenerator
from agentic.agent.planner.task_executor import TaskExecutor
from agentic.agent.planner.validation import TaskValidator
from agentic.agent.planner.cache import CacheManager

In [3]:
# | export
class DynamicTaskExecutor:
    """Production-ready dynamic task execution system"""
    
    def __init__(self, agent=None, console=None, model: str = None):
        # Load model from config if not provided
        if model is None:
            model_config = get_model_config()
            model = model_config.get('name', 'qwen3:8b')
        self.model = model
        
        # Use provided agent or create new one
        if agent is not None:
            self.agent = agent
        else:
            config = AgentConfig(
                name="dynamic_executor",
                instructions="You are an AI agent that executes tasks systematically. Analyze requirements, break down complex problems, and provide detailed technical responses with proper implementation."
            )
            self.agent = Agent(config)
        
        # Use provided console or create new one
        self.console = console if console is not None else Console()
        
        # Initialize components
        self.breakdown_generator = ProjectBreakdownGenerator(self.agent, self.console)
        self.task_generator = TaskGenerator(self.agent, self.console)
        self.task_executor = TaskExecutor(self.agent, self.console)
        self.validator = TaskValidator(self.console)
        self.cache_manager = CacheManager(self.console)
        
        # Configuration
        self.max_retries = 3
        self.max_tasks = 20
        
        # State
        self.context: Optional[ProjectContext] = None
        self.project_breakdown = None
        self.estimated_total_tasks: int = 0
        self.project_folder: Optional[str] = None
    
    async def execute_project(self, user_request: str) -> ProjectContext:
        """Main execution loop with two-phase approach"""
        self.console.print(f"🚀 Starting dynamic execution for: {user_request}")
        
        # Try to restore from cache
        restored_context, restored_breakdown = self.cache_manager.restore_from_cache(user_request)
        if restored_context and restored_breakdown:
            self.console.print("📦 Restored from cache, continuing execution...")
            self.context = restored_context
            self.project_breakdown = restored_breakdown
            self.estimated_total_tasks = self.breakdown_generator.estimate_total_tasks(self.project_breakdown)
        else:
            # Phase 1: Generate project breakdown
            self.console.print("\n📋 PHASE 1: Generating project breakdown...")
            self.project_breakdown = self.breakdown_generator.generate_project_breakdown(user_request)
            
            if not self.project_breakdown:
                self.console.print("❌ Failed to generate project breakdown")
                return ProjectContext(original_request=user_request, project_status=ExecutionStatus.FAILED)
            
            self.console.print(f"✅ Project breakdown complete")
            
            # Create project folder based on title and move into it
            self.project_folder = self._create_project_folder(self.project_breakdown.title)
            self.console.print(f"📁 Created project folder: {self.project_folder}")
            
            # Re-save breakdown in project folder
            filename = f"{self.project_breakdown.title.lower().replace(' ', '_')}_breakdown.txt"
            if os.path.exists(f"../{filename}"):
                import shutil
                shutil.move(f"../{filename}", filename)
            
            # Initialize project context
            self.context = ProjectContext(original_request=user_request)
            
            # Estimate total tasks from breakdown
            self.estimated_total_tasks = self.breakdown_generator.estimate_total_tasks(self.project_breakdown)
            
            # Save initial state
            self.cache_manager.save_to_cache(self.context, self.project_breakdown)
        
        # Phase 2: Execute tasks one by one
        self.console.print("\n🔄 PHASE 2: Executing tasks dynamically...")
        
        task_counter = len(self.context.execution_history) + 1
        
        while not self._is_project_complete() and task_counter <= self.max_tasks:
            self.console.print(f"\n{'='*60}")
            self.console.print(f"📋 GENERATING TASK {task_counter}")
            self.console.print(f"{'='*60}")
            
            # Generate next task using breakdown context
            task = self.task_generator.generate_next_task(self.context, self.project_breakdown, self.estimated_total_tasks)
            if not task:
                self.console.print("❌ No more tasks to generate. Project complete.")
                break
            
            self.console.print(f"✅ Generated: {task.name}")
            self.console.print(f"🔍 Task ID: {task.id}, Dependencies: {task.dependencies}")
            
            # PRE-EXECUTION INTROSPECTION
            pre_validation = self.validator.introspect_task_planning(task, self.project_breakdown, self.context)
            if not pre_validation.success:
                self.console.print(f"⚠️ Pre-execution validation failed: {pre_validation.feedback}")
                if pre_validation.next_action == "regenerate":
                    self.console.print("🔄 Regenerating task with feedback...")
                    regenerated_task = self.task_generator.regenerate_task_with_feedback(self.context, self.project_breakdown, pre_validation.feedback)
                    if regenerated_task:
                        task = regenerated_task
                        self.console.print("✅ Task regenerated successfully")
                    else:
                        self.console.print("⚠️ Task regeneration failed. Proceeding with original task.")
            
            # Execute task
            task_result = await self.task_executor.execute_task(task)
            
            # Update context
            self.context.execution_history.append(task_result)
            
            if not isinstance(self.context.current_artifacts, list):
                self.context.current_artifacts = []
                
            artifacts = task_result.artifacts_created
            # Ensure artifacts is always a list
            if not isinstance(artifacts, list):
                if isinstance(artifacts, str):
                    artifacts = [artifacts]
                elif artifacts is None or isinstance(artifacts, bool):
                    artifacts = []
                else:
                    artifacts = [str(artifacts)]
            
            self.context.current_artifacts.extend(artifacts)
            
            self.context.total_tasks_completed += 1
            
            # Save state after each task
            self.cache_manager.save_to_cache(self.context, self.project_breakdown)
            
            # Update project status
            if task_result.status == ExecutionStatus.FAILED:
                self.console.print(f"⚠️ Task {task.id} failed. Continuing with next task.")
                # Mark task as failed but don't stop execution
                failed_tasks = getattr(self.context, 'failed_tasks', [])
                failed_tasks.append(task.id)
                self.context.failed_tasks = failed_tasks
            
            task_counter += 1
        
        # Final project status
        failed_tasks = getattr(self.context, 'failed_tasks', [])
        if failed_tasks:
            self.context.project_status = ExecutionStatus.PARTIAL_SUCCESS
            self.console.print(f"⚠️ Project completed with {len(failed_tasks)} failed tasks: {failed_tasks}")
        else:
            self.context.project_status = ExecutionStatus.SUCCESS
            self.console.print("✅ All tasks completed successfully")
        
        # Clear cache on completion
        self.cache_manager.clear_cache()
        
        self.console.print(f"\n🎉 Project execution completed! Generated {len(self.context.execution_history)} tasks.")
        return self.context
    
    def _create_project_folder(self, title: str) -> str:
        """Create project folder based on title"""
        # Sanitize title for folder name - keep it simple and meaningful
        folder_name = re.sub(r'[^\w\s-]', '', title).strip()
        folder_name = re.sub(r'\s+', '-', folder_name).lower()
        
        # If title is too long, take first 2-3 words
        parts = folder_name.split('-')
        if len(parts) > 3:
            folder_name = '-'.join(parts[:3])
        
        # Create folder if it doesn't exist
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        # Change to project directory
        os.chdir(folder_name)
        
        return folder_name
    
    def _is_project_complete(self) -> bool:
        """Determine if project is complete based on progress"""
        if not self.context.execution_history:
            return False
        
        if self.context.total_tasks_completed >= max(self.estimated_total_tasks, 5):
            self.console.print(f"✅ Project completion reached: {self.context.total_tasks_completed}/{self.estimated_total_tasks} tasks")
            return True
        
        completion_percentage = (self.context.total_tasks_completed / max(self.estimated_total_tasks, 1)) * 100
        self.console.print(f"📊 Project progress: {self.context.total_tasks_completed}/{self.estimated_total_tasks} tasks ({completion_percentage:.1f}%)")
        
        return False
    
    def save_execution_report(self, filename: str = None) -> str:
        """Save detailed execution report"""
        if not filename:
            filename = "execution_report.json"
        
        with open(filename, 'w') as f:
            json.dump(self.context.model_dump(), f, indent=2, default=str)
        
        self.console.print(f"📊 Execution report saved to: {filename}")
        return filename



In [6]:
# Example Usage 

executor = DynamicTaskExecutor()

user_request = "Build a web scraping framework with rate limiting and data storage"

# Execute project
result = await executor.execute_project(user_request)



2025-10-05 21:19:42,321 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-05 21:19:42,322 - INFO - Initialized LLM client with model: qwen3:14b


🚀 Starting dynamic execution for: Build a web scraping framework with rate limiting and data storage

📋 PHASE 1: Generating project breakdown...

2025-10-05 21:19:42,832 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, I need to break down the user's request to build a web scraping framework with rate limiting and data storage. Let me start by understanding what the user wants. The main components here are web scraping, rate limiting, and data storage. 

First, the project summary should be a concise overview. I'll mention the key features: web scraping, rate limiting to avoid overloading servers, and data storage for the scraped information.

Now, major phases. The first phase is probably setting up the project structure. That makes sense because before coding, you need a good structure. Phase 1: Project Setup and Configuration. Then Phase 2 would be implementing the core functionalities like the scraper and rate limiting. Phase 3 could be data storage integration and testing. Phase 4 might be deployment and documentation.

For phase tasks, each phase needs specific tasks. Phase 1 tasks: analyze requirements, set up the project s

...

"needs_debate": false

}

✅ Generated: Implement Core Scraping Engine with Async HTTP Client

🔍 Task ID: T002, Dependencies: []

TypeError: 'bool' object is not iterable